In [1]:
!pip install chainer==7.7.0

     |████████████████████████████████| 11.2 MB 6.0 MB/s 
  Created wheel for chainer: filename=chainer-7.7.0-py3-none-any.whl size=967522 sha256=893a4e9117dcf52af6ee62e4ae5981bcb77bfafaad996e5b65946c8b9fc4506a
  Stored in directory: /root/.cache/pip/wheels/4f/c2/ca/e46a1181cc92ebe6e2d117a5cd29da5bcfc8464e00edb2cac0
Successfully built chainer


In [2]:
import chainer

In [3]:
import cv2
import csv
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow # for image display
import numpy as np
import pandas as pd
from skimage import io
from PIL import Image 

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
%cd /content/
!rm -rf hyperface
!git clone https://github.com/alvaro-neira/hyperface.git --branch aneira
%cd hyperface/
!git fetch --all
!git pull origin aneira


/content
Cloning into 'hyperface'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 256 (delta 24), reused 21 (delta 9), pack-reused 218
Receiving objects: 100% (256/256), 8.32 MiB | 10.18 MiB/s, done.
Resolving deltas: 100% (151/151), done.
/content/hyperface
Fetching origin
From https://github.com/alvaro-neira/hyperface
 * branch            aneira     -> FETCH_HEAD
Already up to date.


In [6]:
%cd /content/hyperface/scripts

/content/hyperface/scripts


In [7]:
import chainer

import cv2
import os
import numpy as np

import config

# Disable type check in chainer
import models

os.environ["CHAINER_TYPE_CHECK"] = "0"


def _cvt_variable(v):
    # Convert from chainer variable
    if isinstance(v, chainer.variable.Variable):
        v = v.data
        if hasattr(v, 'get'):
            v = v.get()
    return v


def short_hyperface(config_path, img_path, model_path):
    # Load config
    config.load(config_path)

    # Define a model
    model = models.HyperFaceModel()
    model.train = False
    model.report = False
    model.backward = False

    # Initialize model
    chainer.serializers.load_npz(model_path, model)

    # Setup GPU
    if config.gpu >= 0:
        chainer.cuda.check_cuda_available()
        chainer.cuda.get_device(config.gpu).use()
        model.to_gpu()
        xp = chainer.cuda.cupy
    else:
        xp = np

    # Load image file
    img = cv2.imread(img_path)
    if img is None or img.size == 0 or img.shape[0] == 0 or img.shape[1] == 0:
        exit()
    img = img.astype(np.float32) / 255.0  # [0:1]
    img = cv2.resize(img, models.IMG_SIZE)
    img = cv2.normalize(img, None, -0.5, 0.5, cv2.NORM_MINMAX)
    img = np.transpose(img, (2, 0, 1))

    # Create single batch
    imgs = xp.asarray([img])
    x = chainer.Variable(imgs)  # , volatile=True)

    # Forward
    y = model(x)

    # Chainer.Variable -> np.ndarray
    detection = _cvt_variable(y['detection'])
    genders = _cvt_variable(y['gender'])

    gender = genders[0]

    if gender > 0.5:
        return f"Female, detection={detection[0]}"
    else:
        return f"Male, detection={detection[0]}"

In [9]:
print(short_hyperface('/content/hyperface/scripts/config.json','/content/drive/MyDrive/ai/faces1/tv24horas_2021_12_07_01_face1.png','/content/drive/MyDrive/ai/model_epoch_190'))

Male, detection=0.9918990135192871


In [ ]:
for filename in os.listdir("/content/drive/MyDrive/ai/faces"):
    if not filename.endswith(".png"): 
        continue
    img_path = os.path.join("/content/drive/MyDrive/ai/faces", filename)
    image_swapped = io.imread(f"{img_path}") 
    image_2 = cv2.cvtColor(image_swapped, cv2.COLOR_BGR2RGB)
    cv2_imshow(image_2)
    print(filename)
    print(short_hyperface('/content/hyperface/scripts/config.json',img_path,'/content/drive/MyDrive/ai/model_epoch_190'))
    print('\n\n') 